In [5]:
import pandas as pd
import numpy as np

# =============================================================================
# Configuración inicial
# =============================================================================
intervals = list(range(1, 9))
adjacent = {
    1: [2],
    2: [1, 3],
    3: [2, 4],
    4: [3, 5],
    5: [4, 6],
    6: [5, 7],
    7: [6, 8],
    8: [7]
}

# Calcular no adyacentes y k_i
non_adjacent = {}
k = {}
for i in intervals:
    non_adjacent[i] = [j for j in intervals if j not in adjacent[i] and j != i]
    k[i] = len(non_adjacent[i])


In [11]:

# =============================================================================
# (a) Probabilidades de primer orden
# =============================================================================
pi = {}
for i in intervals:
    # Probabilidad de ser seleccionado segundo
    prob_segundo = sum((1/8) * (1/k[j]) for j in non_adjacent[i])
    pi[i] = round(1/8 + prob_segundo, 6)

# Crear DataFrame para primer orden
df_pi = pd.DataFrame.from_dict(pi, orient='index', columns=['π_i'])
df_pi.index.name = 'Intervalo'
df_pi.reset_index(inplace=True)
df_pi


,Intervalo,π_i
0,1,0.270833
1,2,0.245833
2,3,0.241667
3,4,0.241667
4,5,0.241667
5,6,0.241667
6,7,0.245833
7,8,0.270833


In [13]:

# =============================================================================
# (b) Probabilidades de segundo orden
# =============================================================================
pi_ij = pd.DataFrame(index=intervals, columns=intervals, dtype=float)

for i in intervals:
    for j in intervals:
        if i == j:
            pi_ij.loc[i, j] = pi[i]
        elif j in adjacent[i]:
            pi_ij.loc[i, j] = 0.0
        else:
            pi_ij.loc[i, j] = round((1/8) * (1/k[i] + 1/k[j]), 6)

pi_ij


,1,2,3,4,5,6,7,8
1,0.270833,0.000000,0.045833,0.045833,0.045833,0.045833,0.045833,0.041667
2,0.000000,0.245833,0.000000,0.050000,0.050000,0.050000,0.050000,0.045833
3,0.045833,0.000000,0.241667,0.000000,0.050000,0.050000,0.050000,0.045833
4,0.045833,0.050000,0.000000,0.241667,0.000000,0.050000,0.050000,0.045833
5,0.045833,0.050000,0.050000,0.000000,0.241667,0.000000,0.050000,0.045833
6,0.045833,0.050000,0.050000,0.050000,0.000000,0.241667,0.000000,0.045833
7,0.045833,0.050000,0.050000,0.050000,0.050000,0.000000,0.245833,0.000000
8,0.041667,0.045833,0.045833,0.045833,0.045833,0.045833,0.000000,0.270833


In [15]:

# =============================================================================
# (c) Matriz de covarianzas
# =============================================================================
cov = pd.DataFrame(index=intervals, columns=intervals, dtype=float)

for i in intervals:
    for j in intervals:
        if i == j:
            # Varianza
            cov.loc[i, j] = round(pi[i] * (1 - pi[i]), 6)
        else:
            # Covarianza
            cov.loc[i, j] = round(pi_ij.loc[i, j] - pi[i]*pi[j], 6)

cov

,1,2,3,4,5,6,7,8
1,0.197482,-0.066580,-0.019618,-0.019618,-0.019618,-0.019618,-0.020747,-0.031684
2,-0.066580,0.185399,-0.059410,-0.009410,-0.009410,-0.009410,-0.010434,-0.020747
3,-0.019618,-0.059410,0.183264,-0.058403,-0.008403,-0.008403,-0.009410,-0.019618
4,-0.019618,-0.009410,-0.058403,0.183264,-0.058403,-0.008403,-0.009410,-0.019618
5,-0.019618,-0.009410,-0.008403,-0.058403,0.183264,-0.058403,-0.009410,-0.019618
6,-0.019618,-0.009410,-0.008403,-0.008403,-0.058403,0.183264,-0.059410,-0.019618
7,-0.020747,-0.010434,-0.009410,-0.009410,-0.009410,-0.059410,0.185399,-0.066580
8,-0.031684,-0.020747,-0.019618,-0.019618,-0.019618,-0.019618,-0.066580,0.197482


In [18]:
cov_sum = cov.sum().sum() 
cov_sum = round(cov_sum, 6)
print(f"Suma de covarianzas: {cov_sum}")

Suma de covarianzas: -1e-05


In [20]:
# =============================================================================
# Formatear y guardar resultados
# =============================================================================
import os

# Crear carpeta de resultados si no existe
os.makedirs("resultados", exist_ok=True)
ruta_excel = os.path.join("resultados", "resultados_2_1.xlsx")

# Guardar todos los DataFrames en un mismo archivo Excel con múltiples hojas
with pd.ExcelWriter(ruta_excel) as writer:
    df_pi.to_excel(writer, sheet_name='a) Primer Orden')
    pi_ij.to_excel(writer, sheet_name='b) Segundo Orden')
    cov.to_excel(writer, sheet_name='c) Covarianzas')
